In [1]:
#import libraries
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import spacy
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to C:\Users\Josiah
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Josiah
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.DataFrame(pd.read_csv('Twitter Sentiment/cleaned_text.csv'))

def helper(string):
    temp = []
    for word in string[1:-1].split(','):
        word = word.strip()
        word = word[1:-1]
        if word:
          temp.append(word)  
    return temp
data.filtered_tokens = data.filtered_tokens.apply(helper)

In [4]:
# One hot encoding
categories = list(set([item for sublist in data.filtered_tokens for item in sublist]))
#print(categories)

vectorizer = CountVectorizer(binary=True, vocabulary=categories)
sentences = [' '.join(words) for words in data['filtered_tokens']]

# Fit and transform the sentences
one_hot_encoded = vectorizer.fit_transform(sentences)

# Convert the one-hot encoded data to a DataFrame
encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns = categories)

# use the print function to check if the one hot encoding is correct
#print(encoded_df.iloc[954,:].sum(),sentences[954]) 
print(encoded_df)
#print(categories)


       illustrated  opportunity  gttin  circling  balled  emirgan  understory  \
0                0            0      0         0       0        0           0   
1                0            0      0         0       0        0           0   
2                0            0      0         0       0        0           0   
3                0            0      0         0       0        0           0   
4                0            0      0         0       0        0           0   
...            ...          ...    ...       ...     ...      ...         ...   
99995            0            0      0         0       0        0           0   
99996            0            0      0         0       0        0           0   
99997            0            0      0         0       0        0           0   
99998            0            0      0         0       0        0           0   
99999            0            0      0         0       0        0           0   

       summer  installer  d

In [4]:
data = data.iloc[25000:75000]
encoded_df = encoded_df.iloc[25000:75000]

print(data.shape)
print(encoded_df.shape)

(50000, 8)
(50000, 50661)


josiah added this

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(encoded_df, data['labels'], stratify = data['labels'], test_size=0.2, random_state=42)

In [44]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
test = SelectKBest(score_func=chi2, k=2000)

X_train_new = test.fit_transform(X_train, y_train)

In [45]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)
rf_classifier.fit(X_train_new, y_train)


RandomForestClassifier(n_estimators=10, random_state=42)

In [14]:
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(n_estimators=100, random_state=42)

cross_val_score(clf, X_train_new, y_train, cv=10)

KeyboardInterrupt: 

In [46]:
X_test_new = test.transform(X_test)

In [35]:
print(y_pred[0:100])

[1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1
 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1]


In [47]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = rf_classifier.predict(X_test_new)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# You can also print a classification report for more details
print(classification_report(y_test, y_pred))


Accuracy: 0.72
              precision    recall  f1-score   support

           0       0.72      0.70      0.71      5000
           1       0.71      0.73      0.72      5000

    accuracy                           0.72     10000
   macro avg       0.72      0.72      0.72     10000
weighted avg       0.72      0.72      0.72     10000

